In [1]:
#@title Get a smaller version of the Lakh MIDI Dataset v0.1
%%capture
!wget http://hog.ee.columbia.edu/craffel/lmd/clean_midi.tar.gz
!tar xvf clean_midi.tar.gz
!rm clean_midi.tar.gz

dataset_path = "/content/clean_midi"

In [2]:
!pip install pretty_midi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 39.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 4.1 MB/s eta 0:00:00
  Created wheel for pretty_midi: filename=pretty_midi-0.2.10-py3-none-any.whl size=5592287 sha256=e44f25d894df27d012f96c1c634b866209538e2a0c4d112a4ca0c1e1893bc642
  Stored in directory: /root/.cache/pip/wheels/cd/a5/30/7b8b7f58709f5150f67f98fde4b891ebf0be9ef07a8af49f25
Successfully built pretty_midi


In [3]:
from pathlib import Path

# Paths to the files of the dataset
midi_paths = list(Path("clean_midi").resolve().glob("**/*.mid"))

In [4]:
#@title Install `keras_nlp`
%%capture
!pip install keras_nlp

In [5]:
import keras_nlp.layers as nlp_layers

def create_transformer(vocab_size, seq_len, embedding_dim, num_heads, dff, num_layers):
  # Input
    inputs = tf.keras.Input(shape=(seq_len,))

    # Embedding
    embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)(inputs)

    # Encoder
    encoder = nlp_layers.TransformerEncoder(num_heads=num_heads, intermediate_dim=dff)(embedding)

    # Decoder
    decoder = nlp_layers.TransformerDecoder(num_heads=num_heads, intermediate_dim=dff)(embedding, encoder)

    # Output
    outputs = tf.keras.layers.Dense(vocab_size, activation='softmax')(decoder)

    # Crea il modello
    model = tf.keras.Model(inputs=inputs, outputs=outputs)

    return model, encoder, decoder

## Try Tokenization e Generation

In [6]:
import copy
import pretty_midi

class midi_to_tokens():
    def __init__(self, path, steps_per_beat=12):
        self.steps_per_beat = steps_per_beat
        try:
          self.pm = pretty_midi.PrettyMIDI(path)
        except OSError as e:
          print(f"Error reading MIDI file: {e}")
          return
        self.dbs = self.pm.get_downbeats().tolist() + [self.pm.get_end_time()] # dbs := downbeats
        self.tokens = self._tokenize()

    def __call__(self):
        return ' '.join(self.tokens)

    def _time_to_step(self, time):
        return round(self.pm.time_to_tick(time) / self.pm.resolution * self.steps_per_beat)

    def _event_to_tokens(self, event):
        if event in ('bar', 'beat'):
            return [event]
        elif isinstance(event, pretty_midi.containers.Note):
            return [f'note_{event.pitch}', f'len_{self._time_to_step(event.end) - self._time_to_step(event.start)}']

    def _trim_note(self, note, start, end):
        n = copy.copy(note)
        n.start, n.end = max(n.start, start), min(n.end, end)
        return n

    def _tokenize(self, start_measure=1, end_measure=None):
        start, end = self.dbs[start_measure - 1], self.dbs[end_measure or -1]

        notes = []
        for inst in self.pm.instruments:
            notes += inst.notes
        notes.sort(key=lambda x: (x.start, -x.pitch))

        events = []
        events += [(self._time_to_step(db), 'bar') for db in self.dbs if start <= db < end]
        events += [(self._time_to_step(b), 'beat') for b in set(self.pm.get_beats()) - set(self.dbs) if start <= b < end] # beats without downbeats
        events += [(self._time_to_step(max(n.start, start)), self._trim_note(n, start, end)) for n in notes if start <= n.start < end or start < n.end <= end]
        events.sort(key=lambda x: x[0])

        tokens = []
        last_beat = 0
        for step, event in events:
            if event in ('bar', 'beat'):
                last_beat = step
            if step - last_beat:
                tokens.append(f'pos_{step - last_beat}')
            tokens += self._event_to_tokens(event)

        return tokens

    def measures(self, start_measure=1, end_measure=None):
        return self._tokenize(start_measure, end_measure)

In [7]:
str(midi_paths[8])

'/content/clean_midi/Pet Shop Boys/I Wouldnt Normally Do This Kind of Thing.mid'

In [8]:
import numpy as np
tokens = []
for path in midi_paths[:100]:
  try:
    tokens.append(midi_to_tokens(str(path), steps_per_beat=12).tokens)
  except:
    print(f"Error reading MIDI file: {path}")

/usr/local/lib/python3.10/dist-packages/pretty_midi/pretty_midi.py:100: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  warnings.warn(


Error reading MIDI file: undefined status byte 0xf5
Error reading MIDI file: /content/clean_midi/Pet Shop Boys/Always on My Mind   In My House.3.mid


In [9]:
from sklearn.preprocessing import LabelEncoder
import numpy as np
le = LabelEncoder()
flattened_array = np.concatenate([np.array(sublist).flatten() for sublist in tokens])
le.fit(flattened_array)

LabelEncoder()

In [10]:
transformed_tokens = [le.transform(i) for i in tokens]

In [11]:
start_seq = -1
end_seq = -2
pad_id = -3

start_end_tokens = [np.append(start_seq, np.append(i, end_seq)) for i in transformed_tokens]
max_length = max(len(arr) for arr in start_end_tokens)
padded_tokens = np.array([np.pad(arr, (max_length - len(arr), 0), mode='constant', constant_values=pad_id) for arr in start_end_tokens])

In [12]:
vocab_size = le.classes_.shape[0] + 3
seq_len = max_length
import tensorflow as tf


model, encoder, decoder = create_transformer(vocab_size=vocab_size,
                                             seq_len=seq_len,
                                             embedding_dim=256,
                                             num_heads=8,
                                             dff=1024,
                                             num_layers=6)

In [13]:
new_padded_tokens = padded_tokens + 3
print(new_padded_tokens)

[[  0   0   0 ...   4   3   1]
 [  0   0   0 ... 258  82   1]
 [  0   0   0 ... 251   6   1]
 ...
 [  0   0   0 ...   4   4   1]
 [  0   0   0 ...   4   4   1]
 [  0   0   0 ...   4   4   1]]


In [16]:
# prompt: make standard scaling on this features

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled_tokens = scaler.fit_transform(new_padded_tokens)
scaled_tokens

array([[-0.10101525, -0.10101525, -0.10101525, ..., -0.8912749 ,
        -0.5513515 ,  0.        ],
       [-0.10101525, -0.10101525, -0.10101525, ...,  1.18911113,
         0.91685417,  0.        ],
       [-0.10101525, -0.10101525, -0.10101525, ...,  1.13177765,
        -0.49559685,  0.        ],
       ...,
       [-0.10101525, -0.10101525, -0.10101525, ..., -0.8912749 ,
        -0.53276661,  0.        ],
       [-0.10101525, -0.10101525, -0.10101525, ..., -0.8912749 ,
        -0.53276661,  0.        ],
       [-0.10101525, -0.10101525, -0.10101525, ..., -0.8912749 ,
        -0.53276661,  0.        ]])

In [17]:
normalized_train_x = scaled_tokens[:70]
normalized_val_x = scaled_tokens[70:]

normalized_train_y = np.roll(normalized_train_x, shift=-1, axis=1)
normalized_val_y = np.roll(normalized_val_x, shift=-1, axis=1)


In [18]:
from tensorflow.keras.callbacks import EarlyStopping

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

model.fit(normalized_train_x, normalized_train_y,
          epochs=5,
          validation_data=(normalized_val_x, normalized_val_y),
          callbacks=[early_stopping],
          batch_size=32
          )

model.save("NesGen_v1.keras")

Epoch 1/5


ResourceExhaustedError: Graph execution error:

Detected at node StatefulPartitionedCall defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py", line 37, in <module>

  File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelapp.py", line 619, in start

  File "/usr/local/lib/python3.10/dist-packages/tornado/platform/asyncio.py", line 195, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/usr/local/lib/python3.10/dist-packages/tornado/ioloop.py", line 685, in <lambda>

  File "/usr/local/lib/pyt

In [ ]:
import pretty_midi

class TokensToMidi:
    def __init__(self, tokens, steps_per_beat=12, ticks_per_beat=960, tempo=120):
        self.tokens = tokens
        self.steps_per_beat = steps_per_beat
        self.ticks_per_step = ticks_per_beat // steps_per_beat
        self.tempo = tempo
        self.ticks_per_beat = ticks_per_beat

    def _ticks_to_time(self, ticks):
        return ticks * 60 / (self.tempo * self.ticks_per_beat)

    def generate_midi(self):
        pm = pretty_midi.PrettyMIDI(initial_tempo=self.tempo)
        instrument = pretty_midi.Instrument(program=38)

        time = 0
        last_beat = 0

        i = 0
        while i < len(self.tokens):
            token = self.tokens[i]

            if token == "bar":
                time += self._ticks_to_time(self.ticks_per_step * self.steps_per_beat)
                last_beat = time
            elif token == "beat":
                time = last_beat
                last_beat = time
            elif token.startswith("pos_"):
                position = int(token.split("_")[1])
                time = last_beat + self._ticks_to_time(self.ticks_per_step * position)
            elif token.startswith("note_"):
                pitch = int(token.split("_")[1])
                length_token = self.tokens[i + 1]
                length = int(length_token.split("_")[1])
                duration = self._ticks_to_time(self.ticks_per_step * length)

                note = pretty_midi.Note(
                    velocity=100,
                    pitch=pitch,
                    start=time,
                    end=time + duration
                )
                instrument.notes.append(note)

                i += 1
            i += 1

        pm.instruments.append(instrument)
        pm.write("reconstructed_output.mid")
        return pm

In [ ]:
midi_reconstructor = TokensToMidi(tokens)
midi_reconstructor.generate_midi()